In [1]:
# v1.0.7 - 14/02/2020
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",1000)
import geopandas as gpd

from shapely.geometry import Point
crs = {"init":"epsg:4326"}

# ----------------------------------------------------------------------------------------------------------

import matplotlib.pyplot as plt
plt.style.use("ggplot") # to make matplotlib look better
%matplotlib inline
import seaborn as sns

import plotly
import chart_studio
import cufflinks as cf
cf.set_config_file(offline=True) # to make cufflinks work offline

import pandas_bokeh
pandas_bokeh.output_notebook()

#from plotly import colors.DEFAULT_PLOTLY_COLORS # to know default plotly colors
from plotly import graph_objs as go
BKCF_colors = ['#e32592','#fae700','#34b233','#58585a','#939597','#E0E1DD','#5bcbf5','#fe4540','#c7be71'] # to use with cufflinks
layout = go.Layout(colorway=BKCF_colors) # to use with plain plotly
from plotly import tools as tls
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # to allow plotly offline for jupyter notebooks
from bokeh.plotting import save # to allow saving pandas bokeh plots into html files
import plotly.express as px
import yfinance as yf
from datetime import date
import calendar
# ----------------------------------------------------------------------------------------------------------
import pickle
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>")) # to make jupyter notebook appearence bigger

Loading BokehJS ...

In [2]:
#Save in pickle format
def save_obj(obj, name, path="./obj"):
    if not(os.path.isdir(path)):
        os.mkdir(path)
        print("Creado directorio './obj' donde guardar objeto")
    with open("{}/{}.pkl".format(path,name), "wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
#Load a pickle        
def load_obj(name, path="./obj"):
    with open("{}/{}.pkl".format(path, name), "rb") as f:
        return pickle.load(f)

def plot_metrica_line(df, metrica):
    df = df[['symbol','date','{0}'.format(metrica)]]
    colors = ['orange', 'pink','purple','brown','blue','green','black','grey','yellow','azure','gold']
    symbols = df['symbol'].drop_duplicates().values.tolist()
    if len(symbols)>12:
        print("Introduce como máximo 12 compañías")
    else:
        fig = px.line(df[df['symbol']==symbols[0]], x = df.columns[1], y= df.columns[2], color = 'symbol',
                      color_discrete_sequence=['red'],#line_dash_sequence=['dash'], 
                      title="{0}".format(metrica))
        for i in range(1,len(symbols)):
            fig1 = px.line(df[df['symbol']==symbols[i]], x = df.columns[1], y = df.columns[2], color = 'symbol',
                           color_discrete_sequence=[colors[i-1]])
            fig.add_trace(fig1.data[0])
        return fig
    
def plot_metrica_bar(df, metrica):
    df = df[['symbol','date','{0}'.format(metrica)]]
    colors = ['orange', 'pink','purple','brown','blue','green','black','grey','yellow','azure','gold']
    symbols = df['symbol'].drop_duplicates().values.tolist()
    if len(symbols)>12:
        print("Introduce como máximo 12 compañías")
    else:
        fig = px.bar(df[df['symbol']==symbols[0]], x = df.columns[1], y= df.columns[2], color = 'symbol',
                      color_discrete_sequence=['red'],#line_dash_sequence=['dash'], 
                      title="{0}".format(metrica))
        for i in range(1,len(symbols)):
            fig1 = px.bar(df[df['symbol']==symbols[i]], x = df.columns[1], y = df.columns[2], color = 'symbol',
                           color_discrete_sequence=[colors[i-1]])
            fig.add_trace(fig1.data[0])
        return fig
    
from ipywidgets import interact, IntSlider
from IPython.display import display

def freeze_header(df, num_rows=300, num_columns=10, step_rows=1,step_columns=1):
    """
    Freeze the headers (column and index names) of a Pandas DataFrame. A widget
    enables to slide through the rows and columns.

    Parameters
    ----------
    df : Pandas DataFrame
        DataFrame to display
    num_rows : int, optional
        Number of rows to display
    num_columns : int, optional
        Number of columns to display
    step_rows : int, optional
        Step in the rows
    step_columns : int, optional
        Step in the columns

    Returns
    -------
    Displays the DataFrame with the widget
    """
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])
        
############################Figure 1#####################################
def analysis_compañia(df):
    fig1 = go.Figure()

    fig1.add_trace(go.Bar(x=df['year'],
                    y=df['Revenue'],
                    name='Revenue',
                    marker_color='rgb(55, 83, 109)'))

    fig1.add_trace(go.Bar(x=df['year'],
                    y=df['Net Income'],
                    name='Profit',
                    marker_color='rgb(26, 118, 255)'))

    fig1.add_trace(go.Bar(x=df['year'],
                    y=df['Free Cash Flow'],
                    name='Free Cash Flow',
                    marker_color='green'))

    fig1.update_layout(
        title='Revenue, profit and free cash flow',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='USD (billions)',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.


    ############################Figure 2#####################################
    fig2 = go.Figure()

    fig2.add_trace(go.Bar(x=df['year'],
                    y=df['Revenue Growth'],
                    name='Revenue Growth',
                    marker_color='rgb(55, 83, 109)'))

    #fig2.add_trace(go.Bar(x=df['year'],
                   #y=df['Gross Profit Growth'],
                   #name='Gross Profit Growth',
                   #marker_color='purple'))

    fig2.add_trace(go.Bar(x=df['year'],
                    y=df['Net Income Growth'],
                    name='Net Income Growth',
                    marker_color='rgb(26, 118, 255)'))

    fig2.update_layout(
        title='Growth',
        xaxis_tickfont_size=14,
        yaxis=dict(
           # title='USD (billions)',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    ############################Figure 3#####################################
    fig3 = go.Figure()

    fig3.add_trace(go.Bar(x=df['year'],
                    y=df['currentRatio'],
                    name='Current ratio ',
                    marker_color='rgb(26, 118, 255)'))

    fig3.add_trace(go.Bar(x=df['year'],
                    y=df['quickRatio'],
                    name='Quick ratio',
                    marker_color='rgb(55, 83, 109)'))

    fig3.add_trace(go.Bar(x=df['year'],
                    y=df['debtEquityRatio'],
                    name='Debt/equity',
                    marker_color='rgb(55, 34, 189)'))

    fig3.add_trace(go.Bar(x=df['year'],
                    y=df['Debt/EBITDA'],
                    name='Debt/EBITDA',
                    marker_color='rgb(5, 3, 109)'))

    fig3.update_layout(
        title='Liquidy and solvency',
        xaxis_tickfont_size=14,
        yaxis=dict(
           # title='USD (billions)',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    ############################Figure 4#####################################
    fig4 = go.Figure()

    fig4.add_trace(go.Bar(x=df['year'],
                    y=df['priceToSalesRatio'],
                    name='P/S ',
                    marker_color='rgb(5, 3, 109)'))

    fig4.add_trace(go.Bar(x=df['year'],
                    y=df['priceEarningsRatio'],
                    name='P/E ',
                    marker_color='rgb(26, 118, 255)'))

    fig4.add_trace(go.Bar(x=df['year'],
                    y=df['EV/EBITDA'],
                    name='EV/EBITDA',
                    marker_color='rgb(55, 83, 109)'))

    fig3.update_layout(
        title='Valuation metrics',
        xaxis_tickfont_size=14,
        yaxis=dict(
            #title='USD (billions)',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.
    return fig1,fig2,fig3,fig4

# General analysis

In [3]:
df_metricas = load_obj("df_metricas_1")
df_metricas.index.names = ['my_index']
df_metricas['year'] = pd.DatetimeIndex(df_metricas['date']).year
my_categories = pd.read_excel('./data/my_categories.xlsx') 
df_metricas = df_metricas.merge(my_categories,on="symbol")
df_metricas.index = df_metricas['companyName']
df_metricas['Net Income Growth'] = df_metricas['Net Income Growth']*100
#df_metricas_new = load_obj("df_metricas_new")
#df_metricas = pd.concat([df_metricas,df_metricas_new])
#save_obj(df_metricas,"df_metricas_1")
#df_metricas = load_obj("df_metricas_1")

#reduced = df_metricas.drop_duplicates('symbol')
df_reduced = df_metricas[['symbol','companyName','category','date',
         'Price_now','Market_cap_now','EV_now','Market Capitalization','EV','Number of Shares',
         'Revenue','Gross Profit','EBITDA','Net Income','Free Cash Flow',
         'grossProfitMargin','operatingProfitMargin','netProfitMargin','Free Cash Flow margin',
         'Total_debt','Net_cash','currentRatio','quickRatio','Working Capital','debtRatio','debtEquityRatio','Debt/EBITDA','Net_cash/share','interestCoverage',
         'ROA','ROE','ROIC',
         'Revenue Growth','Gross Profit Growth','Net Income Growth',
         'priceEarningsRatio','EPS Growth','priceEarningsToGrowthRatio','priceToSalesRatio','priceToSalesRatio_now','priceEarningsRatio_now',
         'EV/Revenue','EV/EBITDA']]

In [4]:
#freeze_header(df=df_reduced, num_rows=11)
freeze_header(df=df_reduced.drop_duplicates('symbol').sort_values(by=['category','symbol'],ascending=True), num_columns=5,num_rows=12)

interactive(children=(IntSlider(value=12, description='rows', max=85, min=12, readout=False), IntSlider(value=…

In [5]:
freeze_header(df_metricas[df_metricas['category']=="stalwarts"].drop_duplicates('symbol').sort_values(by="symbol"),num_columns=5,num_rows=10)

interactive(children=(IntSlider(value=10, description='rows', max=18, min=10, readout=False), IntSlider(value=…

In [6]:
df = df_metricas[(df_metricas['symbol']=="GIS")&(df_metricas['year']>=2009)]
f1,f2,f3,f4 =analysis_compañia(df)
f1.show()
f2.show()
f3.show()

In [7]:
f4.show()